In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
df = pd.read_csv('/content/Iris.csv')

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X

In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [7]:
map = {
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
df['Species'] = df['Species'].map(map)

In [38]:
train_X, test_X, train_y, test_y = train_test_split(df[df.columns[0:4]].values,
                                                    df['Species'].values,test_size=0.2)

In [39]:
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())

In [40]:
net = Net()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)


In [41]:

for epoch in range(1000):
    optimizer.zero_grad()
    out = net(train_X)
    loss = criterion(out, train_y)
    loss.backward()
    optimizer.step()
    if epoch%100==0:
      print('epoch ',epoch,'loss ',loss.item())

epoch  0 loss  1.1694625616073608
epoch  100 loss  1.0054978132247925
epoch  200 loss  0.9724069237709045
epoch  300 loss  0.9555079936981201
epoch  400 loss  0.952805757522583
epoch  500 loss  0.9415790438652039
epoch  600 loss  0.9276978969573975
epoch  700 loss  0.9167426228523254
epoch  800 loss  0.9137717485427856
epoch  900 loss  0.9115597009658813


In [42]:
predict_out = net(test_X)
_, predict_y = torch.max(predict_out, 1)
print('prediction accuracy', accuracy_score(test_y.data, predict_y.data))

prediction accuracy 0.7333333333333333


In [43]:
print('macro precision', precision_score(test_y.data, predict_y.data, average='macro'))
print('micro precision', precision_score(test_y.data, predict_y.data, average='micro'))
print('macro recall', recall_score(test_y.data, predict_y.data, average='macro'))
print('micro recall', recall_score(test_y.data, predict_y.data, average='micro'))

macro precision 0.5098039215686274
micro precision 0.7333333333333333
macro recall 0.6666666666666666
micro recall 0.7333333333333333


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
